In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
# Считываем тренировочные данные
train = pd.read_csv("precio-de-portatiles/train.csv")
test = pd.read_csv("precio-de-portatiles/test.csv")

print(f"Train dataset shape: {train.shape}")
print(f"Train_formula dataset shape: {test.shape}")

Train dataset shape: (912, 14)
Train_formula dataset shape: (391, 13)


In [3]:
print("Number of missing values in train datasets:")
print(train.isna().sum())
print(test.isna().sum())

Number of missing values in train datasets:
id                  0
laptop_ID           0
Company             0
Product             0
TypeName            0
Inches              0
ScreenResolution    0
Cpu                 0
Ram                 0
Memory              0
Gpu                 0
OpSys               0
Weight              0
Price_euros         0
dtype: int64
id                  0
laptop_ID           0
Company             0
Product             0
TypeName            0
Inches              0
ScreenResolution    0
Cpu                 0
Ram                 0
Memory              0
Gpu                 0
OpSys               0
Weight              0
dtype: int64


In [4]:
# Полные сведения о каждой переменной в тренировочном наборе данных
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 912 entries, 0 to 911
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id                912 non-null    int64  
 1   laptop_ID         912 non-null    int64  
 2   Company           912 non-null    object 
 3   Product           912 non-null    object 
 4   TypeName          912 non-null    object 
 5   Inches            912 non-null    float64
 6   ScreenResolution  912 non-null    object 
 7   Cpu               912 non-null    object 
 8   Ram               912 non-null    object 
 9   Memory            912 non-null    object 
 10  Gpu               912 non-null    object 
 11  OpSys             912 non-null    object 
 12  Weight            912 non-null    object 
 13  Price_euros       912 non-null    float64
dtypes: float64(2), int64(2), object(10)
memory usage: 99.9+ KB


## Предобработка данных

In [5]:
price = train.Price_euros
train = train.drop(['laptop_ID','id','Product','Price_euros'], axis = 1)

In [6]:
for col in train.columns:
    print(col, train[col].nunique())

Company 19
TypeName 6
Inches 17
ScreenResolution 37
Cpu 102
Ram 9
Memory 36
Gpu 94
OpSys 9
Weight 164


In [17]:
train.head()

,Company,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight
0,Asus,2 in 1 Convertible,12.5,Full HD / Touchscreen 1920x1080,Intel Core M M3-6Y30 0.9GHz,4GB,64GB Flash Storage,Intel HD Graphics 515,Chrome OS,1.2kg
1,Asus,Gaming,17.3,Full HD 1920x1080,AMD Ryzen 1600 3.2GHz,8GB,256GB SSD + 1TB HDD,AMD Radeon RX 580,Windows 10,3.2kg
2,Lenovo,Notebook,15.6,Full HD 1920x1080,Intel Core i7 7500U 2.7GHz,4GB,1TB HDD,Intel HD Graphics 620,Windows 10,1.85kg
3,Dell,Ultrabook,13.3,Quad HD+ / Touchscreen 3200x1800,Intel Core i7 7660U 2.5GHz,16GB,512GB SSD,Intel Iris Plus Graphics 640,Windows 10,1.29kg
4,Lenovo,Notebook,15.6,Full HD 1920x1080,Intel Core i7 6500U 2.5GHz,4GB,1.0TB Hybrid,Intel HD Graphics 520,Windows 7,2.32kg


In [18]:
train.Weight = train.Weight.apply(lambda x: float(x[:-2]))
train.Ram = train.Ram.apply(lambda x: int(x[:-2]))
train['Touchscreen'] = train.ScreenResolution.str.contains('Touchscreen').astype(int)
train.ScreenResolution = train.ScreenResolution.apply(lambda x: x.split(' ')[-1])#.unique()
train['Power'] = train.Cpu.apply(lambda x: x.split(' ')[-1][:-3])#.unique()
train.Cpu = train.Cpu.apply(lambda x: ' '.join(x.split(' ')[:-1]))
#train.Cpu = train.Cpu.apply(lambda x: x[:-7].replace(' ', ''))#.unique()
train['Power'] = train['Power'].astype(float)
train.Memory = train.Memory.apply(lambda x: x.replace('.0TB', '000GB'))
train.Memory = train.Memory.apply(lambda x: x.replace('TB', '000GB'))
train.Memory = train.Memory.apply(lambda x: x.replace('Flash Storage', 'Flash_Storage'))
train.Memory = train.Memory.apply(lambda x: x.split(' +  '))#.unique()
train.Gpu = train.Gpu.apply(lambda x: x.replace('<U+039C>', ''))
train.Gpu = train.Gpu.apply(lambda x: x.replace('Quadro M', 'Quadro_m'))
train.Gpu = train.Gpu.apply(lambda x: x.replace('FirePro W', 'FirePro_W'))
train.Cpu = train.Cpu.apply(lambda x: x.replace('-Series', ''))
train.Cpu = train.Cpu.apply(lambda x: x.replace('Atom ', 'Atom_'))
train.Cpu = train.Cpu.apply(lambda x: x.replace('MX', 'mx'))
train.Cpu = train.Cpu.apply(lambda x: x.replace('RX', 'rx'))
train.Cpu = train.Cpu.apply(lambda x: x.replace('UHD ', 'uhd'))
train.Cpu = train.Cpu.apply(lambda x: x.replace('GTX', 'gtx'))
train.Cpu = train.Cpu.apply(lambda x: x.replace('E ', 'E_'))
train.Cpu = train.Cpu.apply(lambda x: x.replace('Pentium Quad Core', 'pentium_Quad_Core'))
train.Cpu = train.Cpu.apply(lambda x: x.replace('Celeron Quad Core', 'Celeron_Quad_Core'))
train.Cpu = train.Cpu.apply(lambda x: x.replace('Celeron Dual Core', 'Celeron_Dual_Core'))

In [19]:
import re

In [20]:
gpu_cols = ['Intel_G', 'Nvidia_G','Samsung_G', 'AMD_G','ARM', 'Iris', 'Pro', 'Plus', 'HD', 'uhd', 'GT', 'gtx', 'Ti',\
            'Quadro', 'Quadro_M','SLI', 'M', 'mx','R2','R3','R4','R5','R7', 'rx', 'Graphics', 'FirePro_W', 'X',\
            'MP4', 'Series']
GP = pd.DataFrame(data=None, index = train.index, columns=gpu_cols).fillna(0)
for i in range(len(GP)):
    card = train.Gpu.iloc[i]
    for prod in ['Intel','Nvidia', 'AMD']:
        if prod in card:
            GP.loc[i, prod+'_G'] = 1
            card = card.replace(prod+' ','')
            for mod in gpu_cols[3:]:
                if mod in card:
                    GP.loc[i, mod] = 1
                if mod == 'MX':
                    GP.loc[i, 'M'] = 0
    
    series = re.findall(r'\d+', card.split()[-1])
    
    if len(series) != 0:
        if int(series[0]) not in [2,3,4,5,7]:
            GP.loc[i, 'Series'] = str(series[0])
        else:
            GP.loc[i, 'Series'] = 0
    else:
        GP.loc[i, 'Series'] = 0
        
GP = GP.drop('Series', axis = 1)
GP = GP.rename(columns={"M": "M_G"}, errors="raise")

In [21]:
cpu_cols = ['Intel_C', 'AMD_C', 'Samsung_C','E', 'A4', 'A6', 'A9', 'A10', 'A12', 'Ryzen',\
            'FX', 'P', 'E_','Cortex','i3', 'i5', 'i7',  'Celeron_Dual_Core', 'pentium_Quad_Core',\
            'Celeron_Quad_Core', 'Atom_x5', 'Atom_Z','Xeon','Y', 'M', 'U', 'HQ', 'HK','Series']
CP = pd.DataFrame(data=None, index = train.index, columns=cpu_cols).fillna(0)
for i in range(len(CP)):
    card = train.Cpu.iloc[i]
    for prod in ['Intel','Samsung', 'AMD']:
        if prod in card:
            CP.loc[i, prod+'_C'] = 1
            card = card.replace(prod+' ','')
            for mod in cpu_cols[3:]:
                if mod in card:
                    if (mod == 'e') and (prod !='AMD'):
                        CP.loc[i, mod] = 0
                    else:
                        CP.loc[i, mod] = 1
    
    series = re.findall(r'\d+', card.split()[-1])
    
    if len(series) != 0:
        if int(series[0]) not in [2,3,4,5,7]:
            CP.loc[i, 'Series'] = str(series[0])
        else:
            CP.loc[i, 'Series'] = 0
    else:
        CP.loc[i, 'Series'] = 0
CP = CP.drop('Series', axis = 1)

In [22]:
Mem = pd.DataFrame(data=None, index = train.index, columns=['SSD', 'HDD','Flash_Storage','Hybrid']).fillna(0)
for i in range(len(Mem)):
    for elem in train.Memory.iloc[i]:
        for col in Mem.columns:
            if col in elem:
                
                Mem[col].iloc[i] += int(elem.split(' ')[0][:-2])
                if len(train.Memory.iloc[i]) ==3:
                    print(elem)
                    print(col,'===', elem.split(' ')[0][:-2])
train = train.drop(['Cpu','Gpu','Memory'], axis = 1)
train = pd.concat([train, Mem], axis=1)
train = pd.concat([train, GP], axis=1)
train = pd.concat([train, CP], axis=1)


In [23]:
train = pd.get_dummies(train)

In [24]:
train

,Inches,Ram,Weight,Touchscreen,Power,SSD,HDD,Flash_Storage,Hybrid,Intel_G,...,ScreenResolution_3840x2160,OpSys_Android,OpSys_Chrome OS,OpSys_Linux,OpSys_Mac OS X,OpSys_No OS,OpSys_Windows 10,OpSys_Windows 10 S,OpSys_Windows 7,OpSys_macOS
0,12.5,4,1.20,1,0.9,0,0,64,0,1,...,0,0,1,0,0,0,0,0,0,0
1,17.3,8,3.20,0,3.2,256,1000,0,0,0,...,0,0,0,0,0,0,1,0,0,0
2,15.6,4,1.85,0,2.7,0,1000,0,0,1,...,0,0,0,0,0,0,1,0,0,0
3,13.3,16,1.29,1,2.5,512,0,0,0,1,...,0,0,0,0,0,0,1,0,0,0
4,15.6,4,2.32,0,2.5,0,0,0,1000,1,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
907,15.6,4,2.18,0,2.5,0,1000,0,0,0,...,0,0,0,0,0,0,1,0,0,0
908,15.6,6,2.40,0,2.4,0,1000,0,0,0,...,0,0,0,0,0,0,1,0,0,0
909,15.6,8,2.20,0,2.5,128,1000,0,0,0,...,0,0,0,0,0,0,1,0,0,0
910,15.6,4,2.19,0,2.0,0,500,0,0,0,...,0,0,0,0,0,0,1,0,0,0


### **Тестовые данные.**

Обработаем аналогичным образом тестовые данные.

In [25]:
labels = test.id
test = test.drop(['laptop_ID','id','Product'], axis = 1)

In [26]:
test.head()

,Company,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight
0,Lenovo,Ultrabook,14.0,IPS Panel Full HD 1920x1080,Intel Core i5 7200U 2.5GHz,8GB,256GB Flash Storage,Intel HD Graphics 620,Windows 10,1.13kg
1,HP,Notebook,15.6,Full HD 1920x1080,AMD E-Series E2-9000e 1.5GHz,4GB,500GB HDD,AMD Radeon R2,Windows 10,2.1kg
2,Lenovo,Ultrabook,14.0,Full HD 1920x1080,Intel Core i7 6600U 2.6GHz,12GB,512GB SSD,Intel HD Graphics 520,Windows 7,1.4kg
3,HP,Gaming,17.3,IPS Panel Full HD 1920x1080,Intel Core i5 6300HQ 2.3GHz,8GB,128GB SSD + 1TB HDD,Nvidia GeForce GTX 965M,Windows 10,2.75kg
4,Lenovo,Ultrabook,14.0,IPS Panel Full HD 1920x1080,Intel Core i7 7700HQ 2.8GHz,8GB,256GB SSD,Nvidia GeForce GT 940MX,Windows 10,1.96kg


In [27]:
test.Weight = test.Weight.apply(lambda x: float(x[:-2]))
test.Ram = test.Ram.apply(lambda x: int(x[:-2]))
test['Touchscreen'] = test.ScreenResolution.str.contains('Touchscreen').astype(int)
test.ScreenResolution = test.ScreenResolution.apply(lambda x: x.split(' ')[-1])#.unique()
test['Power'] = test.Cpu.apply(lambda x: x.split(' ')[-1][:-3])#.unique()
test.Cpu = test.Cpu.apply(lambda x: ' '.join(x.split(' ')[:-1]))
#train.Cpu = train.Cpu.apply(lambda x: x[:-7].replace(' ', ''))#.unique()
test['Power'] = test['Power'].astype(float)
test.Memory = test.Memory.apply(lambda x: x.replace('.0TB', '000GB'))
test.Memory = test.Memory.apply(lambda x: x.replace('TB', '000GB'))
test.Memory = test.Memory.apply(lambda x: x.replace('Flash Storage', 'Flash_Storage'))
test.Memory = test.Memory.apply(lambda x: x.split(' +  '))#.unique()
test.Gpu = test.Gpu.apply(lambda x: x.replace('<U+039C>', ''))
test.Gpu = test.Gpu.apply(lambda x: x.replace('Quadro M', 'Quadro_M'))
test.Gpu = test.Gpu.apply(lambda x: x.replace('FirePro W', 'FirePro_W'))
test.Cpu = test.Cpu.apply(lambda x: x.replace('-Series', ''))
test.Cpu = test.Cpu.apply(lambda x: x.replace('Atom ', 'Atom_'))
test.Cpu = test.Cpu.apply(lambda x: x.replace('MX', 'mx'))
test.Cpu = test.Cpu.apply(lambda x: x.replace('RX', 'rx'))
test.Cpu = test.Cpu.apply(lambda x: x.replace('UHD ', 'uhd'))
test.Cpu = test.Cpu.apply(lambda x: x.replace('GTX', 'gtx'))
test.Cpu = test.Cpu.apply(lambda x: x.replace('E ', 'E_'))
test.Cpu = test.Cpu.apply(lambda x: x.replace('Pentium Quad Core', 'pentium_Quad_Core'))
test.Cpu = test.Cpu.apply(lambda x: x.replace('Celeron Quad Core', 'Celeron_Quad_Core'))
test.Cpu = test.Cpu.apply(lambda x: x.replace('Celeron Dual Core', 'Celeron_Dual_Core'))

In [28]:
gpu_cols = ['Intel_G', 'Nvidia_G','Samsung_G', 'AMD_G','ARM', 'Iris', 'Pro', 'Plus', 'HD', 'uhd', 'GT', 'gtx', 'Ti',\
            'Quadro', 'Quadro_M','SLI', 'M', 'mx','R2','R3','R4','R5','R7', 'rx', 'Graphics', 'FirePro_W', 'X',\
            'MP4', 'Series']
GP = pd.DataFrame(data=None, index = test.index, columns=gpu_cols).fillna(0)
for i in range(len(GP)):
    card = test.Gpu.iloc[i]
    for prod in ['Intel','Nvidia', 'AMD']:
        if prod in card:
            GP.loc[i, prod+'_G'] = 1
            card = card.replace(prod+' ','')
            for mod in gpu_cols[3:]:
                if mod in card:
                    GP.loc[i, mod] = 1
                if mod == 'MX':
                    GP.loc[i, 'M'] = 0
    
    series = re.findall(r'\d+', card.split()[-1])
    
    if len(series) != 0:
        if int(series[0]) not in [2,3,4,5,7]:
            GP.loc[i, 'Series'] = str(series[0])
        else:
            GP.loc[i, 'Series'] = 0
    else:
        GP.loc[i, 'Series'] = 0
        
GP = GP.drop('Series', axis = 1)
GP = GP.rename(columns={"M": "M_G"}, errors="raise")

In [29]:
cpu_cols = ['Intel_C', 'AMD_C', 'Samsung_C','E', 'A4', 'A6', 'A9', 'A10', 'A12', 'Ryzen',\
            'FX', 'P', 'E_','Cortex','i3', 'i5', 'i7',  'Celeron_Dual_Core', 'pentium_Quad_Core',\
            'Celeron_Quad_Core', 'Atom_x5', 'Atom_Z','Xeon','Y', 'M', 'U', 'HQ', 'HK','Series']
CP = pd.DataFrame(data=None, index = test.index, columns=cpu_cols).fillna(0)
for i in range(len(CP)):
    card = test.Cpu.iloc[i]
    for prod in ['Intel','Samsung', 'AMD']:
        if prod in card:
            CP.loc[i, prod+'_C'] = 1
            card = card.replace(prod+' ','')
            for mod in cpu_cols[3:]:
                if mod in card:
                    if (mod == 'e') and (prod !='AMD'):
                        CP.loc[i, mod] = 0
                    else:
                        CP.loc[i, mod] = 1
    
    series = re.findall(r'\d+', card.split()[-1])
    
    if len(series) != 0:
        if int(series[0]) not in [2,3,4,5,7]:
            CP.loc[i, 'Series'] = str(series[0])
        else:
            CP.loc[i, 'Series'] = 0
    else:
        CP.loc[i, 'Series'] = 0
CP = CP.drop('Series', axis = 1)

In [30]:
Mem = pd.DataFrame(data=None, index = test.index, columns=['SSD', 'HDD','Flash_Storage','Hybrid']).fillna(0)
for i in range(len(Mem)):
    for elem in test.Memory.iloc[i]:
        for col in Mem.columns:
            if col in elem:
                
                Mem[col].iloc[i] += int(elem.split(' ')[0][:-2])
test = test.drop(['Cpu','Gpu','Memory'], axis = 1)
test = pd.concat([test, Mem], axis=1)
test = pd.concat([test, GP], axis=1)
test = pd.concat([test, CP], axis=1)
test = pd.get_dummies(test)

In [31]:
test

,Inches,Ram,Weight,Touchscreen,Power,SSD,HDD,Flash_Storage,Hybrid,Intel_G,...,ScreenResolution_3200x1800,ScreenResolution_3840x2160,OpSys_Chrome OS,OpSys_Linux,OpSys_Mac OS X,OpSys_No OS,OpSys_Windows 10,OpSys_Windows 10 S,OpSys_Windows 7,OpSys_macOS
0,14.0,8,1.13,0,2.5,0,0,256,0,1,...,0,0,0,0,0,0,1,0,0,0
1,15.6,4,2.10,0,1.5,0,500,0,0,0,...,0,0,0,0,0,0,1,0,0,0
2,14.0,12,1.40,0,2.6,512,0,0,0,1,...,0,0,0,0,0,0,0,0,1,0
3,17.3,8,2.75,0,2.3,128,1000,0,0,0,...,0,0,0,0,0,0,1,0,0,0
4,14.0,8,1.96,0,2.8,256,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
386,17.3,8,2.54,0,2.0,0,1000,0,0,1,...,0,0,0,0,0,0,1,0,0,0
387,17.3,6,2.65,0,2.4,0,2000,0,0,0,...,0,0,0,0,0,0,1,0,0,0
388,14.0,8,1.60,0,2.8,256,0,0,0,1,...,0,0,0,0,0,0,1,0,0,0
389,15.6,4,2.40,0,2.4,0,1000,0,0,1,...,0,0,0,0,0,0,1,0,0,0


In [32]:
test.columns, train.columns

(Index(['Inches', 'Ram', 'Weight', 'Touchscreen', 'Power', 'SSD', 'HDD',
        'Flash_Storage', 'Hybrid', 'Intel_G',
        ...
        'ScreenResolution_3200x1800', 'ScreenResolution_3840x2160',
        'OpSys_Chrome OS', 'OpSys_Linux', 'OpSys_Mac OS X', 'OpSys_No OS',
        'OpSys_Windows 10', 'OpSys_Windows 10 S', 'OpSys_Windows 7',
        'OpSys_macOS'],
       dtype='object', length=109),
 Index(['Inches', 'Ram', 'Weight', 'Touchscreen', 'Power', 'SSD', 'HDD',
        'Flash_Storage', 'Hybrid', 'Intel_G',
        ...
        'ScreenResolution_3840x2160', 'OpSys_Android', 'OpSys_Chrome OS',
        'OpSys_Linux', 'OpSys_Mac OS X', 'OpSys_No OS', 'OpSys_Windows 10',
        'OpSys_Windows 10 S', 'OpSys_Windows 7', 'OpSys_macOS'],
       dtype='object', length=114))

In [33]:
for column in test.columns:
    if column not in train.columns:
        print(column)

In [34]:
for column in train.columns:
    if column not in test.columns:
        print(column)
        test[column] = 0

Company_Huawei
ScreenResolution_2160x1440
ScreenResolution_2560x1600
ScreenResolution_2736x1824
OpSys_Android


In [35]:
test

,Inches,Ram,Weight,Touchscreen,Power,SSD,HDD,Flash_Storage,Hybrid,Intel_G,...,OpSys_No OS,OpSys_Windows 10,OpSys_Windows 10 S,OpSys_Windows 7,OpSys_macOS,Company_Huawei,ScreenResolution_2160x1440,ScreenResolution_2560x1600,ScreenResolution_2736x1824,OpSys_Android
0,14.0,8,1.13,0,2.5,0,0,256,0,1,...,0,1,0,0,0,0,0,0,0,0
1,15.6,4,2.10,0,1.5,0,500,0,0,0,...,0,1,0,0,0,0,0,0,0,0
2,14.0,12,1.40,0,2.6,512,0,0,0,1,...,0,0,0,1,0,0,0,0,0,0
3,17.3,8,2.75,0,2.3,128,1000,0,0,0,...,0,1,0,0,0,0,0,0,0,0
4,14.0,8,1.96,0,2.8,256,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
386,17.3,8,2.54,0,2.0,0,1000,0,0,1,...,0,1,0,0,0,0,0,0,0,0
387,17.3,6,2.65,0,2.4,0,2000,0,0,0,...,0,1,0,0,0,0,0,0,0,0
388,14.0,8,1.60,0,2.8,256,0,0,0,1,...,0,1,0,0,0,0,0,0,0,0
389,15.6,4,2.40,0,2.4,0,1000,0,0,1,...,0,1,0,0,0,0,0,0,0,0


In [36]:
#check repeating
for col in train.columns:
    
    try:
        print(train[col].dtype)
    except Exception:
        print()
        print(col)
        print('ERROR') #PRO, M
        print()

float64
int64
float64
int64
float64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8


In [37]:
train.columns = train.columns.str.replace(' ', '')
test.columns = test.columns.str.replace(' ', '')

In [38]:
from flaml import AutoML

In [39]:
automl = AutoML()

In [41]:
automl.fit(train, price, task="regression",metric='mae',time_budget=500)

[flaml.automl: 08-06 15:43:19] {2427} INFO - task = regression
[flaml.automl: 08-06 15:43:19] {2429} INFO - Data split method: uniform
[flaml.automl: 08-06 15:43:19] {2432} INFO - Evaluation method: cv
[flaml.automl: 08-06 15:43:19] {2551} INFO - Minimizing error metric: mae
[flaml.automl: 08-06 15:43:19] {2691} INFO - List of ML learners in AutoML Run: ['lgbm', 'rf', 'xgboost', 'extra_tree', 'xgb_limitdepth']
[flaml.automl: 08-06 15:43:19] {2993} INFO - iteration 0, current learner lgbm
[flaml.automl: 08-06 15:43:19] {3126} INFO - Estimated sufficient time budget=577s. Estimated necessary time budget=4s.
[flaml.automl: 08-06 15:43:19] {3173} INFO -  at 0.1s,	estimator lgbm's best error=433.4111,	best estimator lgbm's best error=433.4111
[flaml.automl: 08-06 15:43:19] {2993} INFO - iteration 1, current learner lgbm
[flaml.automl: 08-06 15:43:19] {3173} INFO -  at 0.2s,	estimator lgbm's best error=433.4111,	best estimator lgbm's best error=433.4111
[flaml.automl: 08-06 15:43:19] {2993} 

[flaml.automl: 08-06 15:43:23] {3173} INFO -  at 3.4s,	estimator xgboost's best error=267.6089,	best estimator lgbm's best error=188.6085
[flaml.automl: 08-06 15:43:23] {2993} INFO - iteration 36, current learner extra_tree
[flaml.automl: 08-06 15:43:23] {3173} INFO -  at 3.5s,	estimator extra_tree's best error=207.0179,	best estimator lgbm's best error=188.6085
[flaml.automl: 08-06 15:43:23] {2993} INFO - iteration 37, current learner rf
[flaml.automl: 08-06 15:43:23] {3173} INFO -  at 3.7s,	estimator rf's best error=240.8859,	best estimator lgbm's best error=188.6085
[flaml.automl: 08-06 15:43:23] {2993} INFO - iteration 38, current learner rf
[flaml.automl: 08-06 15:43:23] {3173} INFO -  at 3.8s,	estimator rf's best error=226.2739,	best estimator lgbm's best error=188.6085
[flaml.automl: 08-06 15:43:23] {2993} INFO - iteration 39, current learner lgbm
[flaml.automl: 08-06 15:43:23] {3173} INFO -  at 3.9s,	estimator lgbm's best error=188.6085,	best estimator lgbm's best error=188.608

[flaml.automl: 08-06 15:43:28] {3173} INFO -  at 8.9s,	estimator extra_tree's best error=199.3708,	best estimator lgbm's best error=187.0264
[flaml.automl: 08-06 15:43:28] {2993} INFO - iteration 74, current learner extra_tree
[flaml.automl: 08-06 15:43:28] {3173} INFO -  at 9.1s,	estimator extra_tree's best error=196.0236,	best estimator lgbm's best error=187.0264
[flaml.automl: 08-06 15:43:28] {2993} INFO - iteration 75, current learner lgbm
[flaml.automl: 08-06 15:43:28] {3173} INFO -  at 9.2s,	estimator lgbm's best error=187.0264,	best estimator lgbm's best error=187.0264
[flaml.automl: 08-06 15:43:28] {2993} INFO - iteration 76, current learner lgbm
[flaml.automl: 08-06 15:43:29] {3173} INFO -  at 9.4s,	estimator lgbm's best error=187.0264,	best estimator lgbm's best error=187.0264
[flaml.automl: 08-06 15:43:29] {2993} INFO - iteration 77, current learner extra_tree
[flaml.automl: 08-06 15:43:29] {3173} INFO -  at 9.5s,	estimator extra_tree's best error=196.0236,	best estimator lg

[flaml.automl: 08-06 15:43:42] {3173} INFO -  at 23.2s,	estimator lgbm's best error=187.0264,	best estimator xgboost's best error=173.9031
[flaml.automl: 08-06 15:43:42] {2993} INFO - iteration 111, current learner xgboost
[flaml.automl: 08-06 15:43:46] {3173} INFO -  at 26.9s,	estimator xgboost's best error=173.9031,	best estimator xgboost's best error=173.9031
[flaml.automl: 08-06 15:43:46] {2993} INFO - iteration 112, current learner xgb_limitdepth
[flaml.automl: 08-06 15:43:46] {3173} INFO -  at 27.0s,	estimator xgb_limitdepth's best error=204.7947,	best estimator xgboost's best error=173.9031
[flaml.automl: 08-06 15:43:46] {2993} INFO - iteration 113, current learner xgb_limitdepth
[flaml.automl: 08-06 15:43:46] {3173} INFO -  at 27.1s,	estimator xgb_limitdepth's best error=204.7947,	best estimator xgboost's best error=173.9031
[flaml.automl: 08-06 15:43:46] {2993} INFO - iteration 114, current learner xgb_limitdepth
[flaml.automl: 08-06 15:43:46] {3173} INFO -  at 27.2s,	estimato

[flaml.automl: 08-06 15:44:07] {3173} INFO -  at 47.4s,	estimator lgbm's best error=176.2688,	best estimator xgboost's best error=173.9031
[flaml.automl: 08-06 15:44:07] {2993} INFO - iteration 147, current learner lgbm
[flaml.automl: 08-06 15:44:08] {3173} INFO -  at 48.5s,	estimator lgbm's best error=175.9067,	best estimator xgboost's best error=173.9031
[flaml.automl: 08-06 15:44:08] {2993} INFO - iteration 148, current learner lgbm
[flaml.automl: 08-06 15:44:11] {3173} INFO -  at 51.5s,	estimator lgbm's best error=175.9067,	best estimator xgboost's best error=173.9031
[flaml.automl: 08-06 15:44:11] {2993} INFO - iteration 149, current learner lgbm
[flaml.automl: 08-06 15:44:11] {3173} INFO -  at 51.9s,	estimator lgbm's best error=175.9067,	best estimator xgboost's best error=173.9031
[flaml.automl: 08-06 15:44:11] {2993} INFO - iteration 150, current learner rf
[flaml.automl: 08-06 15:44:12] {3173} INFO -  at 53.0s,	estimator rf's best error=189.3837,	best estimator xgboost's best 

...

[flaml.automl: 08-06 15:47:58] {2993} INFO - iteration 333, current learner lgbm
[flaml.automl: 08-06 15:47:59] {3173} INFO -  at 279.8s,	estimator lgbm's best error=168.4019,	best estimator lgbm's best error=168.4019
[flaml.automl: 08-06 15:47:59] {2993} INFO - iteration 334, current learner lgbm
[flaml.automl: 08-06 15:48:00] {3173} INFO -  at 281.1s,	estimator lgbm's best error=168.4019,	best estimator lgbm's best error=168.4019
[flaml.automl: 08-06 15:48:00] {2993} INFO - iteration 335, current learner lgbm
[flaml.automl: 08-06 15:48:02] {3173} INFO -  at 282.8s,	estimator lgbm's best error=168.4019,	best estimator lgbm's best error=168.4019
[flaml.automl: 08-06 15:48:02] {2993} INFO - iteration 336, current learner lgbm
[flaml.automl: 08-06 15:48:04] {3173} INFO -  at 284.5s,	estimator lgbm's best error=168.4019,	best estimator lgbm's best error=168.4019
[flaml.automl: 08-06 15:48:04] {2993} INFO - iteration 337, current learner lgbm
[flaml.automl: 08-06 15:48:05] {3173} INFO -  a

[flaml.automl: 08-06 15:48:47] {3173} INFO -  at 328.1s,	estimator rf's best error=187.5013,	best estimator lgbm's best error=168.4019
[flaml.automl: 08-06 15:48:47] {2993} INFO - iteration 371, current learner xgb_limitdepth
[flaml.automl: 08-06 15:48:48] {3173} INFO -  at 328.8s,	estimator xgb_limitdepth's best error=174.5568,	best estimator lgbm's best error=168.4019
[flaml.automl: 08-06 15:48:48] {2993} INFO - iteration 372, current learner rf
[flaml.automl: 08-06 15:48:48] {3173} INFO -  at 329.2s,	estimator rf's best error=187.5013,	best estimator lgbm's best error=168.4019
[flaml.automl: 08-06 15:48:48] {2993} INFO - iteration 373, current learner lgbm
[flaml.automl: 08-06 15:48:49] {3173} INFO -  at 330.0s,	estimator lgbm's best error=168.4019,	best estimator lgbm's best error=168.4019
[flaml.automl: 08-06 15:48:49] {2993} INFO - iteration 374, current learner lgbm
[flaml.automl: 08-06 15:48:52] {3173} INFO -  at 332.9s,	estimator lgbm's best error=168.4019,	best estimator lgbm

[flaml.automl: 08-06 15:49:34] {2993} INFO - iteration 408, current learner rf
[flaml.automl: 08-06 15:49:34] {3173} INFO -  at 375.0s,	estimator rf's best error=185.8402,	best estimator lgbm's best error=168.4019
[flaml.automl: 08-06 15:49:34] {2993} INFO - iteration 409, current learner lgbm
[flaml.automl: 08-06 15:49:36] {3173} INFO -  at 377.1s,	estimator lgbm's best error=168.4019,	best estimator lgbm's best error=168.4019
[flaml.automl: 08-06 15:49:36] {2993} INFO - iteration 410, current learner xgb_limitdepth
[flaml.automl: 08-06 15:49:37] {3173} INFO -  at 377.6s,	estimator xgb_limitdepth's best error=174.5568,	best estimator lgbm's best error=168.4019
[flaml.automl: 08-06 15:49:37] {2993} INFO - iteration 411, current learner lgbm
[flaml.automl: 08-06 15:49:39] {3173} INFO -  at 379.7s,	estimator lgbm's best error=168.4019,	best estimator lgbm's best error=168.4019
[flaml.automl: 08-06 15:49:39] {2993} INFO - iteration 412, current learner lgbm
[flaml.automl: 08-06 15:49:41] 

[flaml.automl: 08-06 15:50:22] {2993} INFO - iteration 446, current learner lgbm
[flaml.automl: 08-06 15:50:23] {3173} INFO -  at 423.7s,	estimator lgbm's best error=168.4019,	best estimator lgbm's best error=168.4019
[flaml.automl: 08-06 15:50:23] {2993} INFO - iteration 447, current learner lgbm
[flaml.automl: 08-06 15:50:25] {3173} INFO -  at 426.2s,	estimator lgbm's best error=168.4019,	best estimator lgbm's best error=168.4019
[flaml.automl: 08-06 15:50:25] {2993} INFO - iteration 448, current learner lgbm
[flaml.automl: 08-06 15:50:27] {3173} INFO -  at 427.5s,	estimator lgbm's best error=168.4019,	best estimator lgbm's best error=168.4019
[flaml.automl: 08-06 15:50:27] {2993} INFO - iteration 449, current learner xgb_limitdepth
[flaml.automl: 08-06 15:50:28] {3173} INFO -  at 428.4s,	estimator xgb_limitdepth's best error=174.5568,	best estimator lgbm's best error=168.4019
[flaml.automl: 08-06 15:50:28] {2993} INFO - iteration 450, current learner xgb_limitdepth
[flaml.automl: 08

[flaml.automl: 08-06 15:51:11] {2993} INFO - iteration 483, current learner lgbm
[flaml.automl: 08-06 15:51:14] {3173} INFO -  at 474.9s,	estimator lgbm's best error=168.4019,	best estimator lgbm's best error=168.4019
[flaml.automl: 08-06 15:51:14] {2993} INFO - iteration 484, current learner lgbm
[flaml.automl: 08-06 15:51:15] {3173} INFO -  at 475.7s,	estimator lgbm's best error=168.4019,	best estimator lgbm's best error=168.4019
[flaml.automl: 08-06 15:51:15] {2993} INFO - iteration 485, current learner lgbm
[flaml.automl: 08-06 15:51:17] {3173} INFO -  at 477.5s,	estimator lgbm's best error=168.4019,	best estimator lgbm's best error=168.4019
[flaml.automl: 08-06 15:51:17] {2993} INFO - iteration 486, current learner xgboost
[flaml.automl: 08-06 15:51:19] {3173} INFO -  at 479.7s,	estimator xgboost's best error=169.3994,	best estimator lgbm's best error=168.4019
[flaml.automl: 08-06 15:51:19] {2993} INFO - iteration 487, current learner xgboost
[flaml.automl: 08-06 15:51:20] {3173} 

In [42]:
print('Best ML leaner:', automl.best_estimator)
print('Best hyperparmeter config:', automl.best_config)
print('Best r2 on validation data: {0:.4g}'.format(1-automl.best_loss))
print('Training duration of best run: {0:.4g} s'.format(automl.best_config_train_time))

Best ML leaner: lgbm
Best hyperparmeter config: {'n_estimators': 911, 'num_leaves': 29, 'min_child_samples': 2, 'learning_rate': 0.022149687947851947, 'log_max_bin': 9, 'colsample_bytree': 0.3259955828908142, 'reg_alpha': 0.009643976614491349, 'reg_lambda': 0.14986522938850041}
Best r2 on validation data: -167.4
Training duration of best run: 0.4076 s


In [43]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(train, price, test_size=0.4)

print(f'Train dataset size: {X_train.shape}, {y_train.shape}')
print(f'Test dataset size: {X_test.shape}, {y_test.shape}')

Train dataset size: (547, 114), (547,)
Test dataset size: (365, 114), (365,)


In [44]:
from flaml.ml import sklearn_metric_loss_score
print('train r2', '=', 1-sklearn_metric_loss_score('r2', automl.predict(X_train), y_train))
print(' test r2', '=', 1-sklearn_metric_loss_score('r2', automl.predict(X_test), y_test))

train r2 = 0.9851334314614894
 test r2 = 0.9833248020343266


In [45]:
y_pred_test = automl.predict(test)

In [46]:
len(y_pred_test)

391

In [47]:
len(labels)

391

In [48]:
submission = 'precio-de-portatiles/sample_submission.csv'
ans = pd.read_csv(submission)
ans.id = labels
ans.Price_euros = y_pred_test

In [49]:
ans.to_csv("my_submission.csv", index=False)